In [1]:
import numpy as np
import pandas as pd
import scanpy.api as sc
import anndata



import os
import scipy
from scipy import sparse
from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr
import sys

from keras.layers.merge import _Merge
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from functools import partial

import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np

from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model

import tensorflow as tf

from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap import UMAP
import seaborn as sns 
import Adam_prediction 


tf.compat.v1.disable_eager_execution()

/home/emcandrew/anaconda3/envs/tf_latest/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,
Using TensorFlow backend.


In [2]:
scaler = MinMaxScaler(feature_range=(-1, 1))

exp_mat = pd.read_csv("Gan_input_3_mg.csv", sep='\t', index_col = 0)
exp_mat = exp_mat.T

input_matrix = np.genfromtxt('Gan_input_3_mg.csv', skip_header=1)

input_matrix = input_matrix.T
input_matrix.shape


input_matrix = np.delete(input_matrix, (0), axis=0)

scaler.fit(input_matrix)
input_matrix = scaler.transform(input_matrix)


validation = input_matrix[np.random.choice(input_matrix.shape[0], 500, replace=False)]


i  = np.random.randint(0, input_matrix[0].shape, 500)
validation = input_matrix[i]
input_matrix = np.delete(input_matrix, i, 0)

In [3]:
cell = input_matrix[1].shape
out_dim = cell[0]

In [4]:
Generated_cells = []

In [5]:

class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    def _merge_function(self, inputs):
        alpha = K.random_uniform((32, 1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

class WGANGP():
    def __init__(self):
        self.cell_shape = 2774,
        self.latent_dim = 100
        self.out_dim = out_dim

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        optimizer = Adam(beta_1=0.9, beta_2=0.999)
        
        self.generator = self.build_generator()
        self.critic = self.build_critic()
        
        # Freeze generator's layers while training critic
        self.generator.trainable = False
        
        real_cell = Input(shape=self.cell_shape)
        
        z_disc = Input(shape=(self.latent_dim,))
        # Generate image based of noise (fake sample)
        fake_cell = self.generator(z_disc)
        
        fake = self.critic(fake_cell)
        valid = self.critic(real_cell)
        
        
        # Construct weighted average between real and fake images
        interpolated_cell = RandomWeightedAverage()([real_cell, fake_cell])
        # Determine validity of weighted sample
        validity_interpolated = self.critic(interpolated_cell)
        
         # Use Python partial to provide loss function with additional
        # 'averaged_samples' argument
        partial_gp_loss = partial(self.gradient_penalty_loss,
                          averaged_samples=interpolated_cell)
        partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

        self.critic_model = Model(inputs=[real_cell, z_disc],
                            outputs=[valid, fake, validity_interpolated])
        self.critic_model.compile(loss=[self.wasserstein_loss,
                                              self.wasserstein_loss,
                                              partial_gp_loss],
                                        optimizer=optimizer,
                                        loss_weights=[1, 1, 10])
        
    
        # For the generator we freeze the critic's layers
        self.critic.trainable = False
        self.generator.trainable = True
        
        
        z_gen = Input(shape=(self.latent_dim,))
        # Generate images based of noise
        cell = self.generator(z_gen)
        # Discriminator determines validity
        valid = self.critic(cell)
        # Defines generator model
        self.generator_model = Model(z_gen, valid)
        self.generator_model.compile(loss=self.wasserstein_loss, optimizer=optimizer)
        
    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
        """
        Computes gradient penalty based on prediction and weighted real / fake samples
        """
        gradients = K.gradients(y_pred, averaged_samples)[0]
        # compute the euclidean norm by squaring ...
        gradients_sqr = K.square(gradients)
        #   ... summing over the rows ...
        gradients_sqr_sum = K.sum(gradients_sqr,
                                  axis=np.arange(1, len(gradients_sqr.shape)))
        #   ... and sqrt
        gradient_l2_norm = K.sqrt(gradients_sqr_sum)
        # compute lambda * (1 - ||grad||)^2 still for each single sample
        gradient_penalty = K.square(1 - gradient_l2_norm)
        # return the mean as loss over all the batch samples
        return K.mean(gradient_penalty)
    
    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)
    
    
    def build_generator(self) :
  
      # Defines the model
      model = Sequential()
      # Adds a dense layer of 64 neurons with input_dim equal to z_dim
      model.add(Dense(128, input_dim = self.latent_dim))
      model.add(LeakyReLU(alpha=0.2))
      model.add(Dense(200))
      model.add(LeakyReLU(alpha=0.2))
      # Adds another fully connected layer - output layer 
      model.add(Dense(out_dim, activation = "tanh"))
      #model.add(Reshape(cell))


      model.summary()

      noise = Input(shape=(self.latent_dim,))
      cell = model(noise)

      return Model(noise, cell)

    
  # Defining the Discrimator network

    def build_critic(self) :
      model = Sequential()
      model.add(Dense(200, input_shape = self.cell_shape))
      model.add(LeakyReLU(alpha=0.2))
      model.add(Dense(200))
      model.add(LeakyReLU(alpha=0.2))
      model.add(Dense(1))
        
      model.summary()
    
      cell = Input(shape=self.cell_shape)
      validity = model(cell)
    
    
      return Model(cell, validity)

    def train(self, epochs, batch_size, sample_interval=50):

       # Load the dataset
       X_train = input_matrix
       
       
       # Adversarial ground truths
       valid = -np.ones((batch_size, 1))
       fake =  np.ones((batch_size, 1))
       dummy = np.zeros((batch_size, 1)) # Dummy gt for gradient penalty
       for epoch in range(epochs):

           for _ in range(self.n_critic):

               # ---------------------
               #  Train Discriminator
               # ---------------------

               # Select a random batch of images
               idx = np.random.randint(0, X_train.shape[0], batch_size)
               real_cells = X_train[idx]
               # Sample generator input
               noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
               # Train the critic
               d_loss = self.critic_model.train_on_batch([real_cells, noise],
                                                               [valid, fake, dummy])
               
           g_loss = self.generator_model.train_on_batch(noise, valid)

           # Plot the progress
           print ("%d [D loss: %f] [G loss: %f]" % (epoch, d_loss[0], g_loss))

           # If at save interval => save generated image samples
           if epoch % sample_interval == 0:
               gen_cell = self.generate_cells(5000)
               Generated_cells.append(gen_cell)
                

    def generate_cells(self, number_cells):
        z_dim = 100
        num_cells = number_cells
        noise = np.random.normal(0, 1, (1, z_dim))

        # predcts using enerator network.
        gen_cell = self.generator.predict(noise)

        # Transform back from MinMax scaling between -1 and 1 (necesscary for tanh activation function) 
        gen_cell = scaler.inverse_transform(gen_cell)


       # Repeat of above procedure inside a loop. 
        for cell in range(num_cells -1 ):

                noise = np.random.normal(0, 1, (1, z_dim))

                gen_cells_tmp = self.generator.predict(noise)

                gen_cells_tmp = scaler.inverse_transform(gen_cells_tmp)

              #  Appends subsequent generated cells to form an array of generated cells
                gen_cell = np.append(gen_cell, gen_cells_tmp, axis=0)

        # Converts array to matrice object
        gen_cell = np.asmatrix(gen_cell)

        # converts matrice to pandas dataframe
        gen_cell = pd.DataFrame(gen_cell)

        # Loop to create cell names (theres deinitely a better way to do this)
        cell_names = []
        for i in range(num_cells ):
            cell_names.append("cell-{}".format(i +1))


        # Creates final dataframe with gene names and cell IDs
        gen_cell = pd.DataFrame(data=gen_cell.values, columns=exp_mat.columns, index = cell_names)

        return gen_cell


In [6]:

if __name__ == '__main__':
    wgan = WGANGP()
    wgan.train(epochs=2000200, batch_size=32, sample_interval=1000)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               12928     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               25800     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2774)              557574    
Total params: 596,302
Trainable params: 596,302
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_2"
___________

136 [D loss: -0.495677] [G loss: 3.460676]
137 [D loss: -0.555540] [G loss: 3.252442]
138 [D loss: -0.393297] [G loss: 2.751371]
139 [D loss: -0.377166] [G loss: 1.964214]
140 [D loss: -0.321868] [G loss: 0.358031]
141 [D loss: -0.466852] [G loss: -3.383605]
142 [D loss: -0.543566] [G loss: -5.119133]
143 [D loss: -0.635210] [G loss: -4.708014]
144 [D loss: -0.599138] [G loss: -4.712065]
145 [D loss: -0.257075] [G loss: -2.486699]
146 [D loss: -0.399372] [G loss: 2.890929]
147 [D loss: -0.738079] [G loss: 4.683127]
148 [D loss: -0.615516] [G loss: 4.963731]
149 [D loss: -0.526340] [G loss: 4.162879]
150 [D loss: -0.547504] [G loss: 4.368967]
151 [D loss: -0.686193] [G loss: 4.614023]
152 [D loss: -0.241872] [G loss: 3.294920]
153 [D loss: -0.234452] [G loss: -0.920781]
154 [D loss: -0.812972] [G loss: -4.651597]
155 [D loss: -0.487975] [G loss: -5.128673]
156 [D loss: -0.214559] [G loss: -4.530251]
157 [D loss: -0.334929] [G loss: -4.963236]
158 [D loss: -0.387821] [G loss: -3.795529]


325 [D loss: -0.500771] [G loss: -2.514434]
326 [D loss: -0.454881] [G loss: -0.808106]
327 [D loss: -0.453121] [G loss: -0.602917]
328 [D loss: -0.462201] [G loss: 0.043224]
329 [D loss: -0.450166] [G loss: 0.480156]
330 [D loss: -0.424331] [G loss: 1.724393]
331 [D loss: -0.322298] [G loss: 1.403390]
332 [D loss: -0.314639] [G loss: 0.417666]
333 [D loss: -0.296321] [G loss: 0.707364]
334 [D loss: -0.218041] [G loss: 1.195524]
335 [D loss: -0.150308] [G loss: 0.485117]
336 [D loss: -0.033833] [G loss: -0.036182]
337 [D loss: 0.031915] [G loss: 0.202473]
338 [D loss: -0.105859] [G loss: -0.511158]
339 [D loss: 0.085738] [G loss: -1.643910]
340 [D loss: -0.281535] [G loss: -1.469513]
341 [D loss: -0.358668] [G loss: -3.152005]
342 [D loss: -0.502329] [G loss: -3.345238]
343 [D loss: -0.362477] [G loss: -1.445671]
344 [D loss: -0.525444] [G loss: -0.890177]
345 [D loss: -0.407884] [G loss: -0.769052]
346 [D loss: -0.407664] [G loss: -0.427863]
347 [D loss: -0.255724] [G loss: -0.172485]

514 [D loss: -0.267324] [G loss: 3.538019]
515 [D loss: -0.256322] [G loss: 3.858101]
516 [D loss: -0.170450] [G loss: 3.860827]
517 [D loss: -0.213955] [G loss: 3.475588]
518 [D loss: -0.227494] [G loss: 2.711134]
519 [D loss: -0.304416] [G loss: 1.275714]
520 [D loss: -0.307772] [G loss: 0.798643]
521 [D loss: -0.259053] [G loss: -1.503916]
522 [D loss: -0.435054] [G loss: -3.021573]
523 [D loss: -0.289935] [G loss: -2.439484]
524 [D loss: -0.366586] [G loss: -3.130893]
525 [D loss: -0.330972] [G loss: -3.129252]
526 [D loss: -0.234244] [G loss: -2.179559]
527 [D loss: -0.333177] [G loss: -2.193502]
528 [D loss: -0.269001] [G loss: -2.771853]
529 [D loss: -0.199594] [G loss: -2.430846]
530 [D loss: -0.223605] [G loss: -0.870697]
531 [D loss: -0.196261] [G loss: 1.786092]
532 [D loss: -0.297754] [G loss: 3.352488]
533 [D loss: -0.246570] [G loss: 3.440042]
534 [D loss: -0.276168] [G loss: 3.001544]
535 [D loss: -0.193145] [G loss: 2.569302]
536 [D loss: -0.137538] [G loss: 2.668286]
5

703 [D loss: -0.200849] [G loss: -0.371631]
704 [D loss: -0.209210] [G loss: 1.387494]
705 [D loss: -0.337239] [G loss: 2.863958]
706 [D loss: -0.265307] [G loss: 3.232568]
707 [D loss: -0.418414] [G loss: 4.242431]
708 [D loss: -0.224219] [G loss: 4.441246]
709 [D loss: -0.396278] [G loss: 5.453243]
710 [D loss: -0.129784] [G loss: 5.388738]
711 [D loss: -0.062011] [G loss: 5.251106]
712 [D loss: -0.041497] [G loss: 4.404979]
713 [D loss: -0.221332] [G loss: 2.696689]
714 [D loss: -0.145253] [G loss: 0.104085]
715 [D loss: -0.220011] [G loss: -1.839261]
716 [D loss: -0.199229] [G loss: -2.862343]
717 [D loss: -0.265688] [G loss: -3.398923]
718 [D loss: -0.445805] [G loss: -4.350595]
719 [D loss: -0.231451] [G loss: -4.240123]
720 [D loss: -0.396115] [G loss: -4.613891]
721 [D loss: -0.389375] [G loss: -4.750751]
722 [D loss: -0.182783] [G loss: -4.451836]
723 [D loss: -0.175701] [G loss: -2.772802]
724 [D loss: -0.167920] [G loss: -1.636263]
725 [D loss: -0.187512] [G loss: 0.106210]


...

8293 [D loss: -0.132874] [G loss: -2.423488]
8294 [D loss: -0.066991] [G loss: -2.151675]
8295 [D loss: -0.098525] [G loss: -1.616956]
8296 [D loss: -0.001953] [G loss: -0.431136]
8297 [D loss: -0.075875] [G loss: 0.473749]
8298 [D loss: -0.031376] [G loss: 1.814386]
8299 [D loss: -0.113686] [G loss: 2.759705]
8300 [D loss: -0.064187] [G loss: 2.191705]
8301 [D loss: -0.096534] [G loss: 1.398294]
8302 [D loss: -0.123871] [G loss: 0.847901]
8303 [D loss: -0.098104] [G loss: 0.424410]
8304 [D loss: -0.056919] [G loss: -0.351470]
8305 [D loss: -0.082305] [G loss: -0.601783]
8306 [D loss: -0.118871] [G loss: 0.013406]
8307 [D loss: -0.057233] [G loss: 0.880460]
8308 [D loss: -0.088081] [G loss: 1.510601]
8309 [D loss: -0.089003] [G loss: 1.361198]
8310 [D loss: -0.069032] [G loss: 0.773096]
8311 [D loss: -0.087096] [G loss: 0.430901]
8312 [D loss: -0.081176] [G loss: 0.856366]
8313 [D loss: -0.126961] [G loss: 1.473600]
8314 [D loss: -0.207597] [G loss: 2.220186]
8315 [D loss: -0.037461] [

8478 [D loss: -0.061969] [G loss: -1.700984]
8479 [D loss: -0.028419] [G loss: -1.272565]
8480 [D loss: -0.157919] [G loss: -0.950796]
8481 [D loss: -0.074177] [G loss: -1.029309]
8482 [D loss: -0.048342] [G loss: -0.966666]
8483 [D loss: -0.153234] [G loss: 0.204904]
8484 [D loss: -0.110306] [G loss: 1.235449]
8485 [D loss: -0.080824] [G loss: 1.446433]
8486 [D loss: -0.210743] [G loss: 1.835276]
8487 [D loss: -0.167957] [G loss: 2.395997]
8488 [D loss: -0.157009] [G loss: 3.241622]
8489 [D loss: -0.085857] [G loss: 3.740460]
8490 [D loss: -0.029277] [G loss: 4.204500]
8491 [D loss: 0.023252] [G loss: 4.011590]
8492 [D loss: -0.028125] [G loss: 3.117769]
8493 [D loss: -0.075302] [G loss: 2.309686]
8494 [D loss: -0.062581] [G loss: 1.587347]
8495 [D loss: -0.134518] [G loss: 1.163424]
8496 [D loss: -0.022409] [G loss: 0.453085]
8497 [D loss: -0.014400] [G loss: 0.213568]
8498 [D loss: -0.130390] [G loss: -0.198140]
8499 [D loss: 0.032167] [G loss: -1.095278]
8500 [D loss: -0.125221] [G

8663 [D loss: -0.153493] [G loss: -3.494568]
8664 [D loss: 0.044852] [G loss: -3.691146]
8665 [D loss: -0.035520] [G loss: -2.817742]
8666 [D loss: -0.074056] [G loss: -1.556816]
8667 [D loss: -0.041435] [G loss: -0.308753]
8668 [D loss: -0.046506] [G loss: -0.466762]
8669 [D loss: -0.121752] [G loss: -0.442707]
8670 [D loss: -0.114599] [G loss: 0.207287]
8671 [D loss: -0.092763] [G loss: 1.401902]
8672 [D loss: -0.094189] [G loss: 2.168379]
8673 [D loss: -0.182845] [G loss: 2.780510]
8674 [D loss: -0.167335] [G loss: 3.348718]
8675 [D loss: -0.009422] [G loss: 3.497444]
8676 [D loss: -0.124890] [G loss: 3.683158]
8677 [D loss: -0.087053] [G loss: 3.063831]
8678 [D loss: -0.092876] [G loss: 2.993927]
8679 [D loss: -0.106980] [G loss: 3.076054]
8680 [D loss: -0.153893] [G loss: 3.175246]
8681 [D loss: -0.046036] [G loss: 2.778764]
8682 [D loss: -0.060401] [G loss: 1.939612]
8683 [D loss: -0.031965] [G loss: 2.194251]
8684 [D loss: -0.109484] [G loss: 2.182266]
8685 [D loss: 0.000201] [G

8848 [D loss: -0.166193] [G loss: -2.920920]
8849 [D loss: -0.052127] [G loss: -2.821283]
8850 [D loss: -0.224604] [G loss: -2.258452]
8851 [D loss: 0.057379] [G loss: -2.726515]
8852 [D loss: 0.011717] [G loss: -2.719465]
8853 [D loss: -0.057745] [G loss: -1.873334]
8854 [D loss: -0.017545] [G loss: -2.018367]
8855 [D loss: 0.032220] [G loss: -1.290173]
8856 [D loss: 0.016357] [G loss: 0.237390]
8857 [D loss: -0.028797] [G loss: 1.278507]
8858 [D loss: -0.076170] [G loss: 2.433333]
8859 [D loss: -0.031609] [G loss: 2.998250]
8860 [D loss: -0.121422] [G loss: 2.466043]
8861 [D loss: -0.139744] [G loss: 2.931915]
8862 [D loss: -0.030021] [G loss: 3.828058]
8863 [D loss: -0.253853] [G loss: 3.742304]
8864 [D loss: -0.099678] [G loss: 3.338021]
8865 [D loss: -0.034407] [G loss: 2.740138]
8866 [D loss: 0.074247] [G loss: 3.198127]
8867 [D loss: -0.071009] [G loss: 3.822837]
8868 [D loss: -0.011165] [G loss: 3.178288]
8869 [D loss: 0.034232] [G loss: 2.707371]
8870 [D loss: -0.037196] [G lo

KeyboardInterrupt: 

In [ ]:
 for dataset in Generated_cells :
     test = dataset
     adata = sc.AnnData(test)
     adata.X
     sc.pp.neighbors(adata)
     sc.tl.umap(adata)
     sc.tl.louvain(adata, resolution=1, key_added='louvain_r1')
     sc.pl.umap(adata, color='louvain_r1')
     sc.tl.rank_genes_groups(adata, 'louvain_r1', method='logreg')
     sc.pl.rank_genes_groups(adata, n_genes=20)
     result = adata.uns['rank_genes_groups']
     groups = result['names'].dtype.names
     pd.DataFrame({group + '_' + key[:1]: result[key][group]
         for group in groups for key in ['names', 'scores']}).head(10)